In [13]:
import pandas as pd
import numpy as np
import string
import json
table = str.maketrans('', '', string.punctuation)
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
Stemmer=SnowballStemmer("english", ignore_stopwords=True)
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
from gensim.parsing.preprocessing import remove_stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import re

In [4]:
def word_count(str):
    counts = dict()
    str=remove_stopwords(str)
    words = word_tokenize(str)
    #removing stop words
    for word in words:
        #word = Stemmer.stem(word)
        word= wordnet_lemmatizer.lemmatize(word)
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1
    return counts

In [7]:
with open('D:/Learning Material/IR/WikiPassageQA/data/raw/WikiPassageQA/document_passages.json') as f:
    data = json.load(f)

In [8]:
df = pd.read_csv("D:/Learning Material/IR/WikiPassageQA/data/raw/extracted_query_data/train_exp.csv")

In [9]:
df

,DocumentID,DocumentName,QID,Question,RelevantPassages
0,672,Evangelicalism.html,3086,What is the role of conversionism in Evangelic...,4
1,359,French_Revolution.html,195,How did the assault on the Bastille the first ...,1
2,359,French_Revolution.html,195,How did the assault on the Bastille the first ...,2
3,285,Albania.html,557,What is the prehistory of Albania?,4
4,579,Central_Powers.html,1508,What significance did Bulgaria have in the end...,14
...,...,...,...,...,...
5555,641,Ancient_Egypt.html,1971,How did conquering of Alexander the great effe...,29
5556,641,Ancient_Egypt.html,1971,How did conquering of Alexander the great effe...,30
5557,740,Amnesty_International.html,1153,What is Amnesty International?,0
5558,740,Amnesty_International.html,1153,What is Amnesty International?,9


In [14]:
docid=str(df.iloc[0][0])
relp = str(df.iloc[0][4])
question=df.iloc[0][3].translate(table).lower()
passage=data[docid][relp].translate(table).lower()
passage=re.sub(r'\n', ' ', passage)

In [15]:
q = word_count(question)
p = word_count(passage)
setA = set(q)
setB = set(p)
print("The overlapping words between question and passage is:- ")
print(setA.intersection(setB))
print("Total number of overlapping words: "+str(len(setA.intersection(setB))))
print("Counts of the overlapping words in question and passage:-")
for item in setA.intersection(setB):
    print("Q count for the word '"+ item +"' is "+str(q[item]))
    print("P count for the word '"+ item +"' is "+str(p[item]))

The overlapping words between question and passage is:- 
{'conversionism', 'evangelicalism'}
Total number of overlapping words: 2
Counts of the overlapping words in question and passage:-
Q count for the word 'conversionism' is 1
P count for the word 'conversionism' is 1
Q count for the word 'evangelicalism' is 1
P count for the word 'evangelicalism' is 1


In [16]:
counts=[]
Qcounts=[]
Pcounts=[]
for i in range(len(df)):
    docid=str(df.iloc[i][0])
    relp = str(df.iloc[i][4])
    question=df.iloc[i][3].translate(table).lower()
    passage=data[docid][relp].translate(table).lower()
    passage=re.sub(r'\n', ' ', passage)
    q = word_count(question)
    p = word_count(passage)
    setA = set(q)
    setB = set(p)
    intersect = setA.intersection(setB)
    resQ = {key: q[key] for key in intersect} 
    resP = {key: p[key] for key in intersect}
    Qcounts.append(resQ)
    Pcounts.append(resP)
    counts.append(len(intersect))

In [17]:
counts = np.array(counts)
counts.mean(),counts.std()

(2.427697841726619, 1.4443473627849468)

In [119]:
list(Qcounts[0]),list(Pcounts[0])

(['conversionism', 'evangelicalism'], ['conversionism', 'evangelicalism'])

In [19]:
# Analyzing for the overlapping words how much they occur in the passages on average. 
means=[]
stds=[]
for i in range(len(Pcounts)):
    if len(list(Pcounts[i]))!=0:
        means.append(np.array(list(Pcounts[i].values())).mean())
        stds.append(np.array(list(Pcounts[i].values())).std())

In [20]:
np.array(means).mean(),np.array(means).std()

(2.8601640719212873, 1.6717011183184332)

In [21]:
np.array(stds).mean(),np.array(stds).std()

(0.9353621364542622, 1.040211955614378)

In [24]:
passage_lengths=[]
for i in range(len(df)):
    docid=str(df.iloc[i][0])
    relp = str(df.iloc[i][4])
    passage=data[docid][relp].translate(table).lower()
    passage=re.sub(r'\n', ' ', passage)
    passage=remove_stopwords(passage)
    words = word_tokenize(passage)
    passage_lengths.append(len(words))

In [32]:
tf = np.array(list(Pcounts[0].values()))/passage_lengths[0]

In [81]:
term_freqs=[]
for i in range(len(Pcounts)):
    term_freqs.append(np.array(list(Pcounts[i].values()))/passage_lengths[i])
    

In [92]:
# Analyzing for the overlapping words, the term frequencies in the passages on average. 
means_tf=[]
stds_tf=[]
for i in range(len(term_freqs)):
    if len(list(term_freqs[i]))!=0:
        means_tf.append(term_freqs[i].mean())
        stds_tf.append(term_freqs[i].std())

In [94]:
np.array(means_tf).mean(),np.array(means_tf).std()


(0.036780430648495246, 0.020723265726558412)

In [95]:
np.array(stds_tf).mean(),np.array(stds_tf).std()

(0.011620920957570592, 0.012404180835918452)

In [37]:
allpassages=[]
for i in data.keys():
    for j in data[i].keys():
        allpassages.append(data[i][j])

In [63]:
counts = dict()      
for dicts in Pcounts:
    for word in list(dicts.keys()):        
        count=0
        for passages in allpassages:
            if word in passages:
                count+=1
                continue
        counts[word]=count        

In [67]:
counts["evangelicalism"]

10

In [68]:
import pickle

with open('DocumentFreq.pickle', 'wb') as handle:
    pickle.dump(counts, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [75]:
evangelicalismidf=np.log(50612/(counts["evangelicalism"]+1))

In [77]:
tf[0]*evangelicalismidf

0.138263093582144

In [96]:
term_freqs[0] 

array([0.01639344, 0.01639344])

In [109]:
termfreq_idf=[]
for i,j in zip(term_freqs,Pcounts):
    templ=[]
    for k,l in zip(i,j.keys()):
        templ.append(k*np.log(50612/(counts[l]+1)))
        
    termfreq_idf.append(np.array(templ))    

In [110]:
termfreq_idf[0]

array([0.16620978, 0.13826309])

In [113]:
# Analyzing for the overlapping words how much they occur with respect to term freq_idf in the passages on average. 
means_tf_idf=[]
stds_tf_idf=[]
for i in range(len(termfreq_idf)):
    if len(list(termfreq_idf[i]))!=0:
        means_tf_idf.append(termfreq_idf[i].mean())
        stds_tf_idf.append(termfreq_idf[i].std())

In [114]:
np.array(means_tf_idf).mean(),np.array(means_tf_idf).std()

(0.22857354643418254, 0.16851839428618678)

In [115]:
np.array(stds_tf_idf).mean(),np.array(stds_tf_idf).std()

(0.0927228723529515, 0.10314854542203701)